<a href="https://colab.research.google.com/github/thejfry2/203_final/blob/main/203finalcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install PyTDC

In [ ]:
pip install rdkit torch_geometric torch-scatter torch-sparse torch-cluster torch torchvision pandas scikit-learn


  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 34.3 MB/s eta 0:00:00
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
ERROR: Operation cancelled by user


In [ ]:

!pip install torch-geometric
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-cluster
!pip install rdkit-pypi

  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... canceled
ERROR: Operation cancelled by user
  Using cached torch_cluster-1.6.3.tar.gz (54 kB)
  Preparing metadata (setup.py) ... canceled
ERROR: Operation cancelled by user


In [ ]:
from tdc.single_pred import Tox
from rdkit import Chem
from rdkit.Chem import rdmolops
import torch
from torch_geometric.data import Data
import pandas as pd
# Loading the hERG dataset and spliting it
data = Tox(name='hERG_Karim')
split = data.get_split()

# Combining all data into one dataframe
herg_df = pd.concat([split['train'], split['valid'], split['test']], ignore_index=True)

# this function to convert SMILES to PyTorch Geometric graph
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: #this is for if there is an invalid
        return None

    # Adjacency matrix for edges
    adjacency_matrix = rdmolops.GetAdjacencyMatrix(mol)
    edge_index = torch.tensor(adjacency_matrix.nonzero(), dtype=torch.long)

    # Node features
    node_features = torch.tensor(
        [
            [
                atom.GetAtomicNum(),         # 1. Atomic number
                atom.GetDegree(),            # 2. Degree
                atom.GetHybridization(),     # 3. Hybridization (as enum)
                atom.GetFormalCharge(),      # 4. Formal charge
                atom.GetIsAromatic(),        # 5. Aromaticity
                atom.GetExplicitValence(),   # 6. Explicit valence
                atom.GetImplicitValence(),   # 7. Implicit valence
                atom.GetMass(),              # 8. Atomic mass
                atom.IsInRing()              # 9. Ring membership
            ]
            for atom in mol.GetAtoms()
        ],
        dtype=torch.float
    )

    return Data(x=node_features, edge_index=edge_index)

# Converting all SMILES strings to molecular graphs
molecular_graphs = []
for smiles in herg_df['Drug']:
    graph = smiles_to_graph(smiles)
    if graph is not None:
        molecular_graphs.append(graph)

#a little validation
print(f"Number of molecular graphs: {len(molecular_graphs)}")
print("First molecular graph:")
print(molecular_graphs[0])

Found local copy...
Loading...
Done!


Number of molecular graphs: 13445
First molecular graph:
Data(x=[34, 9], edge_index=[2, 76])


In [ ]:
import torch
from torch.nn import Linear, BCEWithLogitsLoss
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
from torch.optim import Adam

# Defining the GNN model to handle 9 features
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(9, 64)  # Input features: 9, output: 64
        self.conv2 = GCNConv(64, 128)
        self.fc1 = Linear(128, 64)
        self.fc2 = Linear(64, 1)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = global_mean_pool(x, batch)  # Global pooling, I read that this lowers dimentions and makes stuff faster later
        x = self.fc1(x).relu()
        x = self.fc2(x)  # Raw logits
        return x

# Recreating the dataset with 9 features per node
def create_dataset(smiles_list, labels):
    graphs = []
    for smiles, label in zip(smiles_list, labels):
        graph = smiles_to_graph(smiles)  # Updated graph creation function
        if graph is not None:
            graph.y = torch.tensor([label], dtype=torch.float)  # Attach label
            graphs.append(graph)
    return graphs

# Creating a DataLoader for the training set here
train_graphs = create_dataset(split['train']['Drug'], split['train']['Y'])
train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

# This initialises model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GNN().to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = BCEWithLogitsLoss()  # Binary Cross-Entropy with logits

# here is the training loop
for epoch in range(50):  # number of epochs I got maximum performance with 600 but it took almomst half an hour
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)  # Raw logits
        loss = loss_fn(out.squeeze(), batch.y.float())  # Ensure labels are floats
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

print("Training complete!")

Epoch 1, Loss: 205.0672
Epoch 2, Loss: 204.5013
Epoch 3, Loss: 204.5034
Epoch 4, Loss: 204.1079
Epoch 5, Loss: 202.4177
Epoch 6, Loss: 199.7622
Epoch 7, Loss: 195.8640
Epoch 8, Loss: 192.9314
Epoch 9, Loss: 189.3262
Epoch 10, Loss: 188.8357
Epoch 11, Loss: 188.9424
Epoch 12, Loss: 186.3290
Epoch 13, Loss: 186.8683
Epoch 14, Loss: 185.8632
Epoch 15, Loss: 185.2677
Epoch 16, Loss: 184.6518
Epoch 17, Loss: 183.9421
Epoch 18, Loss: 184.0465
Epoch 19, Loss: 183.6033
Epoch 20, Loss: 183.6465
Epoch 21, Loss: 182.7588
Epoch 22, Loss: 182.2604
Epoch 23, Loss: 181.9398
Epoch 24, Loss: 181.2295
Epoch 25, Loss: 181.5088
Epoch 26, Loss: 180.6974
Epoch 27, Loss: 179.8195
Epoch 28, Loss: 179.5336
Epoch 29, Loss: 178.3975
Epoch 30, Loss: 178.5228
Epoch 31, Loss: 178.1142
Epoch 32, Loss: 176.5974
Epoch 33, Loss: 176.7529
Epoch 34, Loss: 176.0216
Epoch 35, Loss: 176.6597
Epoch 36, Loss: 175.8415
Epoch 37, Loss: 175.5564
Epoch 38, Loss: 175.3463
Epoch 39, Loss: 174.8145
Epoch 40, Loss: 174.2982
Epoch 41,

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

# Creating a DataLoader for the validation set here
valid_graphs = create_dataset(split['valid']['Drug'], split['valid']['Y'])
valid_loader = DataLoader(valid_graphs, batch_size=32, shuffle=True)

# Function to test the model
def test_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            preds = (out.squeeze() > 0.5).int()  # Convert logits to binary predictions
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)

    return accuracy, f1, recall

# this section evaluates the model on the validation set
valid_accuracy, valid_f1, valid_recall = test_model(model, valid_loader)
print(f"Validation Set Accuracy: {valid_accuracy:.4f}")
print(f"Validation Set F1 Score: {valid_f1:.4f}")
print(f"Validation Set Recall: {valid_recall:.4f}")

Validation Set Accuracy: 0.6823
Validation Set F1 Score: 0.6421
Validation Set Recall: 0.5794


In [ ]:
print(out.squeeze().detach().cpu().numpy()[:10])  # Check a sample of predictions

[-3.7118025  0.7999965 10.232997  -2.783877 ]
